In [5]:
#Import dependencies.
import pandas as pd 
import gmaps
import requests
import numpy as np
#Import API key.
from config import g_key
#Configure gmaps API key.
gmaps.configure(api_key=g_key)

In [4]:
#Storing CSV file into a DataFrame.
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ostrovnoy,RU,68.0531,39.5131,46.49,79,90,21.14,overcast clouds
1,1,Lebu,CL,-37.6167,-73.6500,50.43,95,100,15.17,light rain
2,2,Benguela,AO,-12.5763,13.4055,71.40,81,77,1.79,broken clouds
3,3,College,US,64.8569,-147.8028,36.59,88,20,3.44,few clouds
4,4,Kavieng,PG,-2.5744,150.7967,84.20,72,80,11.74,broken clouds


In [6]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [7]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Benguela,AO,-12.5763,13.4055,71.40,81,77,1.79,broken clouds
4,4,Kavieng,PG,-2.5744,150.7967,84.20,72,80,11.74,broken clouds
9,9,Bengkulu,ID,-3.8004,102.2655,80.83,82,66,7.27,light rain
15,15,Hithadhoo,MV,-0.6000,73.0833,82.56,75,99,19.75,overcast clouds
18,18,Wanning,CN,18.8003,110.3967,82.44,83,48,8.70,scattered clouds
19,19,Progreso,MX,21.2833,-89.6667,73.51,85,1,8.50,clear sky
27,27,El Vigia,VE,8.6219,-71.6506,74.59,91,100,3.40,overcast clouds
30,30,Butaritari,KI,3.0707,172.7902,81.97,77,12,15.55,few clouds
31,31,Avarua,CK,-21.2078,-159.7750,75.25,73,75,11.50,broken clouds
32,32,Airai,TL,-8.9266,125.4092,74.53,55,97,3.06,overcast clouds


In [8]:
#Drop any empty rows.
clean_travel_cities_df = preferred_cities_df.dropna()
clean_travel_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Benguela,AO,-12.5763,13.4055,71.40,81,77,1.79,broken clouds
4,4,Kavieng,PG,-2.5744,150.7967,84.20,72,80,11.74,broken clouds
9,9,Bengkulu,ID,-3.8004,102.2655,80.83,82,66,7.27,light rain
15,15,Hithadhoo,MV,-0.6000,73.0833,82.56,75,99,19.75,overcast clouds
18,18,Wanning,CN,18.8003,110.3967,82.44,83,48,8.70,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
629,629,Surt,LY,31.2089,16.5887,75.24,66,19,11.12,light rain
632,632,Lufilufi,WS,-13.8745,-171.5986,80.38,78,33,20.78,scattered clouds
633,633,Gasa,PS,31.5000,34.4667,80.89,62,3,5.66,clear sky
637,637,Conceicao Do Araguaia,BR,-8.2578,-49.2647,78.46,40,92,4.05,overcast clouds


In [11]:
hotel_df = clean_travel_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
#Create empty column for Hotel Name.
hotel_df["Hotel Name"] = " "
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Benguela,AO,71.40,broken clouds,-12.5763,13.4055,
4,Kavieng,PG,84.20,broken clouds,-2.5744,150.7967,
9,Bengkulu,ID,80.83,light rain,-3.8004,102.2655,
15,Hithadhoo,MV,82.56,overcast clouds,-0.6000,73.0833,
18,Wanning,CN,82.44,scattered clouds,18.8003,110.3967,


In [12]:
# Hotel search parameters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Benguela,AO,71.40,broken clouds,-12.5763,13.4055,Hotel Praia Morena
4,Kavieng,PG,84.20,broken clouds,-2.5744,150.7967,Nusa Island Retreat
9,Bengkulu,ID,80.83,light rain,-3.8004,102.2655,Grage Hotel Bengkulu
15,Hithadhoo,MV,82.56,overcast clouds,-0.6000,73.0833,Scoop Guest House
18,Wanning,CN,82.44,scattered clouds,18.8003,110.3967,Paradise Hotel
...,...,...,...,...,...,...,...
629,Surt,LY,75.24,light rain,31.2089,16.5887,فندق المدينة
632,Lufilufi,WS,80.38,scattered clouds,-13.8745,-171.5986,Leuaina Beach Resort and Spa
633,Gasa,PS,80.89,clear sky,31.5000,34.4667,رابطة الفنانين الفلسطينيين
637,Conceicao Do Araguaia,BR,78.46,overcast clouds,-8.2578,-49.2647,Tarumã Tropical Hotel


In [18]:
#Create the output file (CSV).
csv_hotel_file = "WeatherPy_vacation.csv"
#Export the City_Data into a CSV.
clean_hotel_df.to_csv(csv_hotel_file, index_label="City_ID")

In [19]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

In [20]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Hotel Praia Morena</dd>\n<dt>City</dt><dd>Benguela</dd>\n<dt>Country</dt><dd>AO</dd>\n<dt>Current Weather</dt><dd>broken clouds and 71.4 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Nusa Island Retreat</dd>\n<dt>City</dt><dd>Kavieng</dd>\n<dt>Country</dt><dd>PG</dd>\n<dt>Current Weather</dt><dd>broken clouds and 84.2 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Grage Hotel Bengkulu</dd>\n<dt>City</dt><dd>Bengkulu</dd>\n<dt>Country</dt><dd>ID</dd>\n<dt>Current Weather</dt><dd>light rain and 80.83 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Scoop Guest House</dd>\n<dt>City</dt><dd>Hithadhoo</dd>\n<dt>Country</dt><dd>MV</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 82.56 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Paradise Hotel</dd>\n<dt>City</dt><dd>Wanning</dd>\n<dt>Country</dt><dd>CN</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 82.44 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Playa Linda Hotel</d

In [21]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
2,-12.5763,13.4055
4,-2.5744,150.7967
9,-3.8004,102.2655
15,-0.6000,73.0833
18,18.8003,110.3967
19,21.2833,-89.6667
27,8.6219,-71.6506
30,3.0707,172.7902
31,-21.2078,-159.7750
32,-8.9266,125.4092


In [22]:
#Add marker layer for each city.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig = gmaps.figure(center = (30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))